In [2]:
#load core libraries
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.ops.gen_dataset_ops import map_dataset

#load the data once at the beginning to save time
df = pd.read_parquet("hf://datasets/pcuenq/oxford-pets/data/train-00000-of-00001-ecc2afb43dedd5e0.parquet")

In [3]:
#get a sense of the data structure
df.head()

,path,label,dog,image
0,/data/datasets/magic-ml/oxford-iiit-pet/images...,Siamese,False,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1,/data/datasets/magic-ml/oxford-iiit-pet/images...,Birman,False,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2,/data/datasets/magic-ml/oxford-iiit-pet/images...,shiba inu,True,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3,/data/datasets/magic-ml/oxford-iiit-pet/images...,staffordshire bull terrier,True,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4,/data/datasets/magic-ml/oxford-iiit-pet/images...,basset hound,True,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...


In [30]:
#making a python list from the series (column) in df
images = [img['bytes'] for img in df["image"]]
labels = df['label'].values

IMG_SIZE = (224,224)

def preprocess(image, label):
    image_new = tf.image.decode_jpeg(image, channels=3)
    #resize the image for the models input
    image_new = tf.image.resize(image_new, IMG_SIZE)
    #resize the image to be within the confines of tensors [-1,1]
    image_new = image_new / 255.0
    return image_new, label

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(preprocess)

print(dataset)

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>


In [33]:
import keras
training_size = 0.8
#test size is not needed as we will take whatever is leftover after the split

#split the dataset into subsets for training purposes
split_tuple = tf.keras.utils.split_dataset(dataset = dataset, left_size = training_size, shuffle=True, seed=0)

#having an error on line 6:
#Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

val_test_split = tf.keras.utils.split_dataset(dataset = split_tuple[0], left_size = 0.875, shuffle=False)
train_dataset = val_test_split[0]
validation_dataset = val_test_split[1]
test_dataset = split_tuple[1]

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

Everything Below is untested and in the works still

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:

def data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])
for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

In [12]:

#load the base model
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False
inputs = keras.Input(shape=IMG_SHAPE)